In [1]:
import requests
from bs4 import BeautifulSoup
import re
import itertools
import pandas as pd, numpy as np
import os
import glob
import pathlib
import re
import janome
import jaconv
from janome.tokenizer import Tokenizer
from janome.analyzer import Analyzer
from janome.charfilter import *
import string
import MeCab
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
import unicodedata
from gensim import corpora
from gensim import models
from pprint import pprint

In [2]:
df = pd.read_csv('/work/data/BunshuOnline/yahoo_news.csv', index_col=0)

In [3]:
df.head()

,title,category,text
0,内村航平が新型コロナ陽性,スポーツ,国際体操連盟FIGの渡辺守成会長らが29日オンラインで緊急会見を開き体操男子の種目別鉄棒で...
1,くそたれ地震 不満のみ込む今,地域,死者44人建物被害が5万棟に上った北海道胆振東部地震から2年がたった被害の大半を占める厚真...
2,GoToイート紙切れ 担当者憤り,地域,新型コロナウイルスの影響で打撃を受けた飲食業界の支援策Go To イート事業のプレミアム付...
3,週末 東日本で冷え込み強まる,国内,明日30日金は上空の寒気が日本列島の上空に南下してきます明日夜からは特に関東や東海など東日本...
4,SB周東が12戦連続盗塁 プロ新,スポーツ,ソフトバンクロッテ29日ペイペイドーム ソフトバンク周東が12試合連続盗塁のプロ野球新記録...


In [4]:
mecab = MeCab.Tagger('-d /usr/lib/x86_64-linux-gnu/mecab/dic/mecab-ipadic-neologd/')
wakati = MeCab.Tagger("-Owakati -d /usr/lib/x86_64-linux-gnu/mecab/dic/mecab-ipadic-neologd/")

In [5]:
def MecabMorphologicalAnalysis(path='./text.txt', output_file='wakati.txt', mecab=mecab, file=False):
    mecab_text = ''
    if file:
        with open(path) as f:
            for line in f:
                mecab_text += mecab.parse(line)
        with open(output_file, 'w') as f:
            print(mecab_text, file=f)
    else:
        for path in path.split('\n'):
            mecab_text += mecab.parse(path)
        return mecab_text

In [6]:
texts = []
for i in range(len(df)):
    texts.append(MecabMorphologicalAnalysis(df['text'][i], mecab=wakati))

In [7]:
# リストに変換
sentences = []
for text in texts:
    text_list = text.split(' ')
    sentences.append(text_list)

In [8]:
# まずdocumentsを作り、モデルに渡す。
documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(sentences)]
model = Doc2Vec(documents, vector_size=100, window=5, min_count=1)

In [19]:
model.docvecs.most_similar(positive=[12], topn=1000)

[(36, 0.999007523059845),
 (338, 0.9990039467811584),
 (329, 0.998987078666687),
 (105, 0.9989807605743408),
 (341, 0.9989774823188782),
 (411, 0.9989742040634155),
 (106, 0.9989701509475708),
 (322, 0.998966634273529),
 (435, 0.9989660978317261),
 (93, 0.9989620447158813),
 (473, 0.9989614486694336),
 (154, 0.9989601969718933),
 (187, 0.998958945274353),
 (177, 0.9989567399024963),
 (158, 0.9989553689956665),
 (136, 0.9989525079727173),
 (110, 0.9989477396011353),
 (335, 0.9989434480667114),
 (294, 0.9989397525787354),
 (11, 0.9989392757415771),
 (434, 0.9989376068115234),
 (334, 0.9989367723464966),
 (402, 0.9989356994628906),
 (220, 0.9989355206489563),
 (310, 0.9989351034164429),
 (342, 0.9989314079284668),
 (210, 0.9989312887191772),
 (415, 0.998927652835846),
 (285, 0.9989267587661743),
 (484, 0.9989247918128967),
 (130, 0.9989238381385803),
 (231, 0.998920202255249),
 (413, 0.9989194869995117),
 (52, 0.9989190697669983),
 (249, 0.9989187121391296),
 (403, 0.9989173412322998),
 (

In [14]:
model.docvecs.most_similar(negative=[12], topn=5)

[(362, -0.9874406456947327),
 (30, -0.9886828660964966),
 (72, -0.9912722706794739),
 (278, -0.9940394759178162),
 (404, -0.9941489696502686)]

In [20]:
model.docvecs[12]

array([ 0.0352571 ,  0.14141539,  0.08895981, -0.10715698,  0.03389508,
       -0.01070596,  0.05212108, -0.05998946, -0.05487895,  0.04995334,
        0.07087918, -0.06375228, -0.03679444, -0.00585587, -0.02627558,
       -0.00026111,  0.10017037,  0.02736529, -0.04584696,  0.03769891,
        0.04576377, -0.07784439,  0.02639231, -0.00338707, -0.00624113,
       -0.0027817 , -0.0681078 , -0.00924025, -0.09670955,  0.08324454,
       -0.10398418, -0.17092592, -0.04109628,  0.00758105,  0.11387365,
        0.04450253, -0.00753651, -0.03119906,  0.05833049, -0.04894487,
        0.03622339,  0.00814112,  0.06453288,  0.02015019, -0.00821397,
        0.19210249,  0.03847395, -0.03644057, -0.09826296,  0.04922669,
        0.02574767, -0.0564045 , -0.01893597,  0.03512792,  0.00105097,
       -0.02214708, -0.1017793 , -0.06355518, -0.03809424,  0.02419831,
       -0.04119074, -0.03727949, -0.05358017,  0.07340679,  0.09368083,
       -0.02452944,  0.02507696,  0.04550331, -0.06830591, -0.02

In [23]:
len(model.docvecs)

496

In [25]:
vecs = []
for i in range(len(model.docvecs)):
    vecs.append(model.docvecs[i])

In [31]:
category = list(df['category'])

In [41]:
df1 = pd.DataFrame(vecs)

In [42]:
df1.insert(0, 'category', category)

In [43]:
df1

,category,0,1,2,3,4,5,6,7,8,...,90,91,92,93,94,95,96,97,98,99
0,スポーツ,0.087332,0.366591,0.230364,-0.290142,0.071419,-0.045537,0.132718,-0.143668,-0.143461,...,-0.129523,-0.056882,-0.031373,-0.162552,-0.144390,0.172448,-0.069078,0.007575,-0.185907,-0.038352
1,地域,0.037272,0.173717,0.107088,-0.133718,0.035155,-0.023503,0.066690,-0.071873,-0.065702,...,-0.063048,-0.022613,-0.017420,-0.080496,-0.074445,0.079186,-0.034293,0.005196,-0.084665,-0.019530
2,地域,0.137399,0.602336,0.376634,-0.472022,0.121876,-0.070285,0.219086,-0.238062,-0.237797,...,-0.204233,-0.089955,-0.055726,-0.271066,-0.244668,0.288515,-0.118685,0.018656,-0.310385,-0.053159
3,国内,0.066954,0.291756,0.187807,-0.227900,0.060894,-0.029280,0.109931,-0.117971,-0.111854,...,-0.105078,-0.044798,-0.023884,-0.133445,-0.114065,0.139010,-0.056173,0.007313,-0.153156,-0.030009
4,スポーツ,0.102108,0.458487,0.290017,-0.360039,0.090577,-0.051486,0.162591,-0.185402,-0.176529,...,-0.160523,-0.076748,-0.046354,-0.199583,-0.188202,0.216386,-0.085842,0.016376,-0.233035,-0.049484
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
491,地域,0.074285,0.343946,0.216986,-0.279011,0.074454,-0.035390,0.124102,-0.136813,-0.140908,...,-0.119925,-0.050206,-0.030793,-0.153712,-0.144015,0.161507,-0.071798,0.011993,-0.177472,-0.032301
492,スポーツ,0.074267,0.312567,0.204726,-0.250485,0.069952,-0.037944,0.115877,-0.130302,-0.129540,...,-0.114431,-0.050312,-0.025346,-0.138429,-0.131895,0.157270,-0.062477,0.008860,-0.162038,-0.032334
493,エンタメ,0.070440,0.331017,0.204093,-0.264739,0.069465,-0.035009,0.128990,-0.129986,-0.133098,...,-0.116929,-0.052106,-0.032333,-0.150320,-0.134057,0.153083,-0.066556,0.007316,-0.168411,-0.033134
494,エンタメ,0.102449,0.474341,0.293014,-0.372542,0.104786,-0.060028,0.178376,-0.188838,-0.190428,...,-0.166020,-0.075839,-0.047465,-0.207599,-0.192509,0.232745,-0.096812,0.015755,-0.239904,-0.050510


In [44]:
import numpy as np, pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
import japanize_matplotlib
from sklearn.decomposition import PCA
from mpl_toolkits.mplot3d import Axes3D

In [45]:
pca = PCA(n_components=2)
pca